# DeepStream과 MQTT를 활용한 실시간 객체 감지 데이터 전송

DeepStream을 활용하여 실시간 객체 감지 후, MQTT를 통해 감지된 데이터를 외부 시스템으로 전송하는 방법을 다룹니다. 이를 통해 IoT 장치, 클라우드 서비스, 데이터베이스 등 다양한 환경에서 실시간 감지 데이터를 활용할 수 있습니다.

## 04-2. MQTT란?
MQTT(Message Queuing Telemetry Transport)는 가벼운 메시징 프로토콜로, IoT 및 실시간 데이터 전송에 적합합니다. DeepStream에서는 감지된 객체 정보를 MQTT 브로커를 통해 퍼블리시하고, 다른 애플리케이션이 이를 구독하여 활용할 수 있습니다.
### 1. 활용 분야
#### **클라우드 연동**
- AWS IoT Core 또는 Azure IoT Hub에 MQTT 메시지를 전송하여 AI 분석 데이터 활용

#### **실시간 알림 시스템**
- 특정 객체 감지 시 알림을 전송하는 시스템 구현

## 04-3. MQTT 환경 설정
### 1. Mosquitto MQTT 브로커 설치
```bash
$ sudo apt update
$ sudo apt install mosquitto mosquitto-clients
```

Mosquitto 서비스 활성화:
```bash
$ sudo systemctl start mosquitto
$ sudo systemctl enable mosquitto
```

### 2. MQTT Python 클라이언트(paho-mqtt) 설치
```bash
$ pip install paho-mqtt
```

## 04-4. DeepStream과 MQTT 연동
### 1. DeepStream 메시지 브로커 설정
DeepStream에서는 `nvds_msgbroker` 플러그인을 사용하여 MQTT로 데이터를 전송할 수 있습니다.

**설정 파일 예시 (`config_mqtt.txt`)**:
```ini
[message-broker]
proto-lib=/opt/nvidia/deepstream/deepstream/lib/libnvds_mqtt_proto.so
conn-str=localhost;1883;deepstream_topic
config-file=config_mqtt.json
```

**MQTT 설정 파일 예시 (`config_mqtt.json`)**:
```json
{
    "mqtt_broker": "localhost",
    "mqtt_port": 1883,
    "topic": "deepstream_topic",
    "client_id": "deepstream_client",
    "qos": 1
}
```

### 2. DeepStream 실행
```bash
$ deepstream-app -c config_mqtt.txt
```

이제 감지된 객체 정보가 MQTT 브로커를 통해 `deepstream_topic`으로 전송됩니다.

## 04-5. MQTT 데이터 수신
### 1. Python MQTT Subscriber 작성
MQTT 데이터를 구독하고 출력하는 간단한 스크립트:
```python
import paho.mqtt.client as mqtt

def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    client.subscribe("deepstream_topic")

def on_message(client, userdata, msg):
    print(f"Received message: {msg.payload.decode()}")

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("localhost", 1883, 60)
client.loop_forever()
```

### 5.2 실행 결과
DeepStream이 감지한 객체 정보가 MQTT를 통해 전송되고, 위의 Python 스크립트가 이를 수신하여 출력합니다.

